## Adding a validation_epoch_end
* You may want to see your accuracy metric every validation epoch end.
* Note that support for validation_epoch_end has been removed in v2.0.0.
* Here is a snippet code:
```
from torchmetrics import Accuracy
  
class LitModel(pl.LightningModule):  
   def __init__(self):
        self.val_accuracy = Accuracy(task="multiclass", num_classes=10)
    
   def validation_step(self, batch, batch_idx):
        ...
        self.val_accuracy.update(preds, y)

        # Calling self.log will surface up scalars for you in TensorBoard
        self.log("val_loss", loss, prog_bar=True)
        self.log("val_acc", self.val_accuracy, prog_bar=True)
        return loss
        
    #Support for `validation_epoch_end` has been removed in v2.0.0.
    def validation_epoch_end(self, validation_step_outputs): 
        avg_loss = torch.stack([x['val_loss'] for x in validation_step_outputs]).mean()
```

In [1]:
import os
import torch
import pytorch_lightning as pl
#import lightning as pl
from torchvision.datasets import MNIST
from torch.utils.data import DataLoader
from torchvision import transforms
from torch import nn
from torch.nn import functional as F

from torchmetrics import Accuracy

class LitModel(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.l1 = nn.Linear(28 * 28, 10)
        self.val_accuracy = Accuracy(task="multiclass", num_classes=10)
        

    def forward(self, x):
        return torch.relu(self.l1(x.view(x.size(0), -1)))

    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.cross_entropy(y_hat, y)
        # logs metrics for each training_step,
        # and the average across the epoch, to the progress bar and logger
        self.log("train_loss", loss, on_step=True, on_epoch=True, prog_bar=True, logger=True)
        return loss
    
    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = F.cross_entropy(y_hat, y)
        preds = torch.argmax(y_hat, dim=1)
        self.val_accuracy.update(preds, y)

        # Calling self.log will surface up scalars for you in TensorBoard
        self.log("val_loss", loss, prog_bar=True, on_step=True, on_epoch=True )
        self.log("val_acc", self.val_accuracy, prog_bar=True, on_step=True, on_epoch=True )
        return {'val_loss': loss}
    
    #Support for `validation_epoch_end` has been removed in v2.0.0.
    def validation_epoch_end(self, validation_step_outputs): 
        avg_loss = torch.stack([x['val_loss'] for x in validation_step_outputs]).mean()
        self.log("avg_val_loss", avg_loss, prog_bar=True)
        return {'avg_val_loss': avg_loss}
    
    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=0.02)

BATCH_SIZE = 128 if torch.cuda.is_available() else 32
#train_loader = DataLoader(MNIST(os.getcwd(), download=True, transform=transforms.ToTensor()))

train_loader = DataLoader(
    MNIST(os.getcwd(), download=True, transform=transforms.ToTensor()), num_workers=4, 
    batch_size = BATCH_SIZE
)

val_loader = DataLoader(
    MNIST(os.getcwd(), download=True, transform=transforms.ToTensor()), num_workers=4,
    batch_size = BATCH_SIZE
)

#mnist_full = MNIST(self.data_dir, train=True, transform=self.transform)
#mnist_train, mnist_val = random_split(mnist_full, [55000, 5000])


trainer = pl.Trainer(
    accelerator="auto",
    max_epochs = 5
)
model = LitModel()
trainer.fit(model, train_loader, val_loader)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/scratch/qualis/miniconda3/envs/lightning/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/logger_connector/logger_connector.py:67: UserWarning: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default
  warning_cache.warn(


NotImplementedError: Support for `validation_epoch_end` has been removed in v2.0.0. `LitModel` implements this method. You can use the `on_validation_epoch_end` hook instead. To access outputs, save them in-memory as instance attributes. You can find migration examples in https://github.com/Lightning-AI/lightning/pull/16520.